In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import os
import sys

In [29]:
sys.path.append(os.path.abspath(os.path.join("../../..")))

In [30]:
import pandas as pd

from notebooks.utils.constants import question_ids_map
from notebooks.utils.simplified_experiment import SimplifiedExperiment

## Threshold for Levenshtein distance


In [31]:
threshold = 80

Retrieve answers file


In [32]:
train_dataset = pd.read_excel("../../aggregated_results/train_dataset.xlsx")

In [33]:
train_dataset

,Lease,lessor_name_correct,effective_date_correct,commencement_date_correct,end_date_correct,option_to_purchase_correct,option_to_extend_correct,option_to_terminate_correct,option_to_terminate_lessor_correct,total_correct,tag
0,78,1,NaN,1.0,0.0,1.0,0.0,0.0,1.0,4,0
1,68,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,6,1
2,76,0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,5,1
3,22,0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,3,0
4,92,1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,4,0
...,...,...,...,...,...,...,...,...,...,...,...
65,7,0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,6,1
66,24,0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,4,0
67,71,0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,4,0
68,56,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,6,1


In [34]:
answers_df = pd.read_excel("../../../data/asc_842/lease_agreements_info/answers.xlsx")
answers_df = answers_df.iloc[train_dataset["Lease"] - 1]
answers_df

,Lease,2,4,25,26,29,32,37,40
77,lease078,"Rehco Holdings, LLC",NaT,2021-12-01,2026-12-01,0.0,0.0,1.0,1.0
67,lease068,STATE OF WYOMING,2017-02-02,2019-11-02,2029-11-01,0.0,0.0,0.0,1.0
75,lease076,TARGET GROUP INC.,2019-12-20,2018-06-28,2019-06-28,0.0,0.0,0.0,0.0
21,lease022,"ARE-MA REGION NO. 75, LLC",2021-12-20,2023-11-30,2035-02-01,0.0,1.0,1.0,1.0
91,lease092,Ever Winland Limited,2021-02-26,2020-12-15,2022-10-02,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
6,lease007,"AR Industrial No. 1, Ltd",2009-02-10,2010-01-31,2020-01-31,0.0,1.0,1.0,1.0
23,lease024,"APT Cowork, LLC",NaT,NaT,NaT,0.0,0.0,1.0,1.0
70,lease071,Steven and Janet Atkinson,2024-04-01,2024-04-01,2029-04-01,0.0,1.0,1.0,1.0
55,lease056,State of Wyoming,2017-02-02,2019-08-02,2029-08-01,0.0,0.0,0.0,1.0


Get questions to send to the AI


In [35]:
question_id = question_ids_map["lessor_name"]

In [36]:
template_string = """What is the name of the lessor in the agreement in the Context? The lessor is also referred to as the "landlord" in certain cases. Return only the exact name of the lessor in the answer_string field. Format any all-caps values in the text to title case. Write the response without using definitional terms. Refer to each entity or concept by its full name or a clear descriptor each time it is mentioned, rather than assigning it a shorthand label or nickname.
Context: {context}
"""

## Experiment 10:

- gpt-4o-mini
- text-embedding-3-large
- structured outputs


In [37]:
experiment = SimplifiedExperiment(
    answers_df=answers_df,
    question_id=question_id,
    threshold=threshold,
    index_name="eafit-text-embedding-3-large",
    lessor_question=True,
    csv_results_filename="results/lessor_name_prompt_tuning_v7.csv",
    template_string=template_string,
    retriever_question="What are the parties involved in this agreement?",
    use_llm_filter=True,
    numbers_list=[str(i).zfill(3) for i in (answers_df.index + 1).to_list()],
)
experiment.run()

Lease: lease078
Answer: Rehco Holdings, LLC
Real answer: Rehco Holdings, LLC
CORRECT
----------------------------------------------------
Lease: lease068
Answer: State Of Wyoming, Acting By And Through Its Board Of Land Commissioners
Real answer: STATE OF WYOMING
INCORRECT
----------------------------------------------------
Lease: lease076
Answer: E. & E. McLaughlin Ltd.
Real answer: TARGET GROUP INC.
INCORRECT
----------------------------------------------------
Lease: lease022
Answer: Landlord
Real answer: ARE-MA REGION NO. 75, LLC
INCORRECT
----------------------------------------------------
Lease: lease092
Answer: Ever Winland Limited
Real answer: Ever Winland Limited
CORRECT
----------------------------------------------------
Lease: lease099
Answer: Landlord
Real answer: Wickfield Phoenix LLC
INCORRECT
----------------------------------------------------
Lease: lease080
Answer: Landlord
Real answer: Ailanthus L.L.C.
INCORRECT
----------------------------------------------------